In [ ]:
!pip install PIL
!pip install matplotlib

In [ ]:
# -*- coding: utf-8 -*-
import json

all_posts = '../posts/posts.json'
list_file_name = '../posts/files.json'

prefix = '{"id":'
prefix_len = len(prefix)
tag_string = 'tag_string":"'
tag_string_len = len(tag_string)

def get_line_info(line):
    first_comma = line.find(',', prefix_len)
    id = line[prefix_len:first_comma]
    tag_string_start = line.find(tag_string, first_comma)
    tag_string_end = line.find('"', tag_string_start + tag_string_len)

    tag_strings = line[tag_string_start + tag_string_len:tag_string_end]
    return id, tag_strings

def get_line_id(line):
    first_comma = line.find(',', prefix_len)
    id = line[prefix_len:first_comma]
    return id

import requests
from PIL import Image
from io import BytesIO
import matplotlib.pyplot as plt

def display_image(image_url):
    data = requests.get(image_url).content
    img = Image.open(BytesIO(data))
    plt.imshow(img)
    plt.axis('off')


def display_multiple_images(image_urls, offset=0):
    if len(image_urls) == 0:
        print('No images to display')
        return
    if len(image_urls) > 12:
        # cut the list to 12 by batch
        for i in range(0, len(image_urls), 12):
            display_multiple_images(image_urls[i:i + 12], offset=i)
        return

    fig = plt.figure(figsize=(20, 20))
    sqrt = int(len(image_urls) ** 0.5)

    columns = min(5, sqrt)
    rows = len(image_urls) // columns + 1
    for i in range(1, columns * rows + 1):
        if i > len(image_urls):
            break
        data = requests.get(image_urls[i - 1]).content
        img = Image.open(BytesIO(data))
        fig.add_subplot(rows, columns, i)
        plt.imshow(img)
        plt.axis('off')
        plt.title(i + offset)
    plt.show()

def match_tag(tag: str, tag_strings: str):
    padding = ' ' + tag + ' '
    padded_tag_strings = ' ' + tag_strings + ' '
    return padded_tag_strings.find(padding) != -1

def match_tags(tags: list, tag_strings: str):
    for tag in tags:
        if tag not in tag_strings:
            return False
    return True



In [ ]:
print(match_tag('a', 'a b c') == True)
print(match_tag('b', 'a b c') == True)
print(match_tag('c', 'a b c') == True)
print(match_tag('e', 'a be c') == False)
print(match_tag('e', 'a be c e') == True)
print(match_tag('d', 'a b c') == False)

print(match_tags(['a', 'b'], '1 a b c ') == True)
print(match_tags(['a', 'b'], '1 a b c d ') == True)
print(match_tags(['a', 'b'], '1 a b ') == True)
print(match_tags(['a', 'e'], '1 a b ') == False)
print(match_tags(['a', 'e'], '1 a b c d e ') == True)
print(match_tags(['a', 'e'], '1 a be c d e f ') == True)
print(match_tags(['a', 'ef'], '1 a be c d ef ') == True)

In [ ]:

line_count = 0
max_count = 10
id_to_found = '7344500'
with open(all_posts, 'r', encoding = 'utf-8') as f:
    for line in f:
        pass
    # last_line
    first_comma = line.find(',', prefix_len)
    id = line[prefix_len:first_comma]
    print(id)




In [ ]:
with open(all_posts, 'r', encoding = 'utf-8') as f:
    line = f.readline()
    print(line)
    print(get_line_info(line))

In [ ]:
# split the file into files each containing 200000 posts, and only keep id and tag_string property
# write the files to disk, add file info to a file list, and write the file list to disk


current_file_piece = 0
current_file_name = f'../posts/posts_{current_file_piece}.json'
current_tag_file_name = f'../posts/posts_{current_file_piece}_tags.json'

current_file = open(current_file_name, 'w', encoding = 'utf-8')
current_tag_file = open(current_tag_file_name, 'w', encoding = 'utf-8')

list_file = open(list_file_name, 'w', encoding = 'utf-8')

line_count = 0
start_id = None
end_id = None
line_count_max = 200000
with open(all_posts, 'r', encoding = 'utf-8') as f:
    for line in f:
        line_count += 1
        if line_count > line_count_max:
            current_file.close()
            current_tag_file.close()

            list_file.write(json.dumps({
                'file': current_file_name,
                'tag_file': current_tag_file_name,
                'start_id': start_id,
                'end_id': end_id,
            }) + '\n')

            current_file_piece += 1
            current_file_name = f'../posts/posts_{current_file_piece}.json'
            current_tag_file_name = f'../posts/posts_{current_file_piece}_tags.json'


            start_id = None
            current_file = open(current_file_name, 'w', encoding = 'utf-8')
            current_tag_file = open(current_tag_file_name, 'w', encoding = 'utf-8')
            line_count = 0

        id, tag_strings = get_line_info(line)
        if start_id is None:
            start_id = id
        end_id = id

        current_file_size = current_file.tell()
        line_binary_len = current_file.write(line)
        current_tag_file.write(f'{{"id":{id},"tag_string":" {tag_strings} ", "offset":[{current_file_size},{line_binary_len}]}}\n')

current_tag_file.close()
current_file.close()
list_file.write(json.dumps({
    'file': current_file_name,
    'tag_file': current_tag_file_name,
    'start_id': start_id,
    'end_id': end_id,
}) + '\n')

list_file.close()

In [ ]:
import threading
from queue import Queue

def search_file_with_tag(tag: list, files_queue, res_queue, thread_id):
    print('start', thread_id)
    while not files_queue.empty():
        file_info = json.loads(files_queue.get())
        # print('searching', file_info['file'])
        current_piece_info = []
        with open('.' + file_info['tag_file'], 'r', encoding = 'utf-8') as tag_file:
            for tag_line in tag_file:
                if match_tags(tag, tag_line):
                    line_info = json.loads(tag_line)
                    # print('found', line_info['id'])
                    current_piece_info.append(line_info)
                    pass

        if len(current_piece_info) == 0:
            files_queue.task_done()
            # print('done', file_info['file'])
            continue

        # print('found', len(current_piece_info), 'lines')

        with open('.' + file_info['file'], 'r', encoding = 'utf-8') as post_file:
            for line_info in current_piece_info:
                # print('seeking', file_info['file'], line_info)
                post_file.seek(line_info['offset'][0])
                line = post_file.read(line_info['offset'][1])
                # print('found line', line)
                res_queue.put(line)

        files_queue.task_done()
        # print('done', file_info['file'])
    print('done all', thread_id)

def map_reduce_search_tag(tag: list, max_count: int = 10, start_id: int = None, end_id: int = None):

    files_queue = Queue()
    res_queue = Queue()
    threads = []

    file_added_count = 0
    with open(list_file_name, 'r', encoding = 'utf-8') as f:
        # search parallel

        for line in f:
            file_info = json.loads(line)
            if start_id is not None and int(file_info['end_id']) < start_id:
                continue
            if end_id is not None and int(file_info['start_id']) > end_id:
                continue

            files_queue.put(line)
            file_added_count += 1

    print('file added', file_added_count)
    updated_tags = [' ' + x + ' ' for x in tag]
    for i in range(8):
        t = threading.Thread(target=search_file_with_tag, args=(updated_tags, files_queue, res_queue, i))
        t.start()
        threads.append(t)

    print('threads started')
    for t in threads:
        t.join()
        print('threads done')

    ret = []

    while not res_queue.empty():
        data = res_queue.get()
        json_data = json.loads(data)
        if start_id is not None and int(json_data['id']) < start_id:
            continue
        if end_id is not None and int(json_data['id']) > end_id:
            continue
        ret.append(json_data)

    return ret

query = [
    'simple_background',
]
search_res = map_reduce_search_tag(query, max_count=40,)
img_idx = 0
for file_info in search_res:
    print(file_info['id'])
    print(file_info['tag_string'])
    if 'large_file_url' in file_info:
        print(img_idx, file_info['large_file_url'])
        img_idx += 1
    else:
        print('no large file url', json.dumps(file_info))
    # preview the image

display_multiple_images([file_info['large_file_url'] for file_info in search_res if 'large_file_url' in file_info])
